In [21]:
"""7-1.seq2seq(data).ipynb"""

!pip install sentencepiece

# Commented out IPython magic to ensure Python compatibility.
# 작업 디렉토리를 변경한다.
# %cd '/content/drive/My Drive/Colab Notebooks'

# Seq2Seq ChatBot : 학습 데이터 모듈
# Google의 Sentencepiece를 이용해서 학습 데이터를 생성한다.
#
# 저작자: 2022.03.29_Hwang 
# 코드 원본 참조 조성현 (blog.naver.com/chunjein)
# copyright: SNS 등에 공개할 때는 출처에 저작자를 명시해 주시기 바랍니다.
# -----------------------------------------------------------------------

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import sentencepiece as spm
import re
import pickle

In [4]:
# 데이터 파일을 읽어온다.
data_df = pd.read_csv('ChatBotData.csv', header=0)
question, answer = list(data_df['Q']), list(data_df['A'])

In [5]:
# 특수 문자를 제거한다.
FILTERS = "([~.,!?\"':;)(])"
question = [re.sub(FILTERS, "", s) for s in question]
answer = [re.sub(FILTERS, "", s) for s in answer]

data_df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
# # 학습 데이터와 시험 데이터를 분리한다.
# que_train, que_test, ans_train, ans_test = train_test_split(question, answer, test_size=0.1, random_state=0)



In [6]:
# que_train[0], ans_train[0]

question[0], answer[0]



('12시 땡', '하루가 또 가네요')

In [8]:
# Sentencepice용 사전을 만들기 위해 que_train + que_test를 저장해 둔다.
data_file = "chatbot_data.txt"
with open(data_file, 'w', encoding='utf-8') as f:
    for sent in question + answer:
        f.write(sent + '\n')  

In [10]:
# Google의 Sentencepiece를 이용해서 vocabulary를 생성한다.
# -----------------------------------------------------
templates= "--input={} \
            --pad_id=0 --pad_piece=<PAD>\
            --unk_id=1 --unk_piece=<UNK>\
            --bos_id=2 --bos_piece=<BOS>\
            --eos_id=3 --eos_piece=<EOS>\
            --model_prefix={} \
            --vocab_size={}"

VOCAB_SIZE = 9000
model_prefix = "chatbot_model"
params = templates.format(data_file, model_prefix, VOCAB_SIZE)

spm.SentencePieceTrainer.Train(params)
sp = spm.SentencePieceProcessor()
sp.Load(model_prefix + '.model')

with open(model_prefix + '.vocab', encoding='utf-8') as f:
    vocab = [doc.strip().split('\t') for doc in f]

word2idx = {k:v for v, [k, _] in enumerate(vocab)}
idx2word = {v:k for v, [k, _] in enumerate(vocab)}



In [11]:
# 학습 데이터를 생성한다. (인코더 입력용, 디코더 입력용, 디코더 출력용)
MAX_LEN = 15    #서브워드 길이
enc_input = []
dec_input = []
dec_output = []

for Q, A in zip(question, answer):
    # Encoder 입력
    enc_i = sp.encode_as_ids(Q)
    enc_input.append(enc_i)

    # Decoder 입력, 출력
    dec_i = [sp.bos_id()]   # <BOS>에서 시작함
    dec_o = []
    for ans in sp.encode_as_ids(A):
        dec_i.append(ans)
        dec_o.append(ans)
    dec_o.append(sp.eos_id())   # Encoder 출력은 <EOS>로 끝남.        
    
    # dec_o는 <EOS>가 마지막에 들어있다. 나중에 pad_sequences()에서 <EOS>가
    # 잘려 나가지 않도록 MAX_LEN 위치에 <EOS>를 넣어준다.
    if len(dec_o) > MAX_LEN:
        dec_o[MAX_LEN] = sp.eos_id()
        
    dec_input.append(dec_i)
    dec_output.append(dec_o)



In [12]:
# 각 문장의 길이를 맞추고 남는 부분에 padding을 삽입한다.
enc_input = pad_sequences(enc_input, maxlen=MAX_LEN, value = sp.pad_id(), padding='post', truncating='post')
dec_input = pad_sequences(dec_input, maxlen=MAX_LEN, value = sp.pad_id(), padding='post', truncating='post')
dec_output = pad_sequences(dec_output, maxlen=MAX_LEN, value = sp.pad_id(), padding='post', truncating='post')



In [18]:
enc_input[1]
                              # 패딩 시작
# array([ 251, 7084,  724, 1598,    0,    0,    0,    0,    0,    0,    0,
#           0,    0,    0,    0], dtype=int32)

array([ 251, 7084,  724, 1598,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [19]:
# 답변
dec_input[1]
#         <BOS>                         <PAD>
# array([   2, 1421,  143, 1465,   91,    0,    0,    0,    0,    0,    0,
#           0,    0,    0,    0], dtype=int32)

array([   2, 1421,  143, 1465,   91,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [20]:
dec_output[1]
#                                 <EOS>
# array([1421,  143, 1465,   91,    3,    0,    0,    0,    0,    0,    0,
#           0,    0,    0,    0], dtype=int32)

array([1421,  143, 1465,   91,    3,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [17]:
# 사전과 학습 데이터를 저장한다.
with open('chatbot_voc.pkl', 'wb') as f:
    pickle.dump([word2idx, idx2word], f, pickle.HIGHEST_PROTOCOL)
# BLEU 평가를 위해 que_test와 ans_test를 저장해 둔다.
with open('chatbot_train.pkl', 'wb') as f:
    pickle.dump([enc_input, dec_input, dec_output], f, pickle.HIGHEST_PROTOCOL)


---> 전처리 끝